# METAUX LOURDS

Seuls l'Arsenic (As) et le Plomb (Pb) sont pris en compte dans la catégorie des métaux lourds.  
La catégorie "métaux lourds" n'est donc pas utilisée et on pourra choisir sur le site As ou Pb.  
Les requêtes sont donc créées en ce sens : une pour chaque substance.

In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 25

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

# Test de requête

In [2]:
%%sql
select * from int__resultats_udi_communes where inseecommune = '07194' limit 5

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdparametresiseeaux,de_partition,valtraduite,categorie,categorie_2,categorie_3,limite_qualite,valeur_sanitaire_1,valeur_sanitaire_2,cdreseau,inseecommune,datetimeprel
00700169098,24D,2020,0.000,pesticide,sub_active,None,0.1,30.0,None,007001388,07194,2020-06-17 10:36:00
00700169098,3HXC,2020,0.000,pesticide,metabolite,pertinent_par_defaut,0.1,None,None,007001388,07194,2020-06-17 10:36:00
00700169098,ACET,2020,0.000,pesticide,sub_active,None,0.1,None,None,007001388,07194,2020-06-17 10:36:00
00700169098,ADET,2020,0.000,pesticide,metabolite,pertinent_par_defaut,0.1,60.0,None,007001388,07194,2020-06-17 10:36:00
00700169098,ADETD,2020,0.000,pesticide,metabolite,pertinent_par_defaut,0.1,60.0,None,007001388,07194,2020-06-17 10:36:00


# Obtention du libellé exact pour les métaux lourds

In [3]:
%%sql
SELECT DISTINCT
    categorie
FROM
    int__resultats_udi_communes
;

Running query in 'duckdb:///../../database/data.duckdb'

categorie
pfas
pesticide
nitrate
cvm
substances_indus
metaux_lourds


# Sélection des résultats de prélèvements pour tous les métaux lourds

In [4]:
%%sql

SELECT
    *
FROM
    int__resultats_udi_communes
WHERE
    categorie = 'metaux_lourds'
LIMIT 5
;

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,de_partition,valtraduite,categorie,categorie_2,categorie_3,limite_qualite,valeur_sanitaire_1,valeur_sanitaire_2,cdreseau,inseecommune,datetimeprel
00100119085,AS,2020,2.000,metaux_lourds,None,None,10.0,13.0,None,001001076,01261,2020-02-14 08:37:00
00100119494,PB,2020,0.000,metaux_lourds,None,None,10.0,None,None,001001019,01033,2020-01-15 10:02:00
00100119495,PB,2020,0.000,metaux_lourds,None,None,10.0,None,None,001000979,01247,2020-01-21 11:53:00
00100119496,PB,2020,0.000,metaux_lourds,None,None,10.0,None,None,001000961,01397,2020-01-22 11:36:00
00100119497,PB,2020,0.000,metaux_lourds,None,None,10.0,None,None,001000958,01173,2020-01-22 13:55:00


# Récupération des cdparametresiseeaux pour identifier Arsenic et Plomb

In [5]:
%%sql

SELECT DISTINCT
    cdparametresiseeaux
FROM
    int__resultats_udi_communes
WHERE
    categorie = 'metaux_lourds'
;

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux
PB
AS


--> AS et PB

# Requête Plomb

In [6]:
%%sql

SELECT
    *
FROM
    int__resultats_udi_communes
WHERE
    categorie = 'metaux_lourds'
    AND cdparametresiseeaux = 'PB'
LIMIT 5
;

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,de_partition,valtraduite,categorie,categorie_2,categorie_3,limite_qualite,valeur_sanitaire_1,valeur_sanitaire_2,cdreseau,inseecommune,datetimeprel
08900131687,PB,2024,0.000,metaux_lourds,None,None,10.0,None,None,089000438,89021,2024-08-16 08:55:00
08900132518,PB,2024,6.000,metaux_lourds,None,None,10.0,None,None,089000441,89025,2024-10-14 11:09:00
08900131968,PB,2024,0.000,metaux_lourds,None,None,10.0,None,None,089000757,89066,2024-09-04 11:29:00
08900131063,PB,2024,0.000,metaux_lourds,None,None,10.0,None,None,089000697,89077,2024-06-17 08:20:00
08900131726,PB,2024,0.000,metaux_lourds,None,None,10.0,None,None,089000473,89080,2024-08-28 08:31:00


## Vérification unicité prélèvement (referenceprel)

In [7]:
%%sql

SELECT DISTINCT de_partition as annee, referenceprel, cdparametresiseeaux, cdreseau, inseecommune, COUNT(*) AS nb_analyses
FROM int__resultats_udi_communes
WHERE categorie = 'metaux_lourds' AND cdparametresiseeaux = 'PB'
GROUP BY annee, referenceprel, cdparametresiseeaux, cdreseau, inseecommune
HAVING COUNT(*) > 1
LIMIT 10

Running query in 'duckdb:///../../database/data.duckdb'

annee,referenceprel,cdparametresiseeaux,cdreseau,inseecommune,nb_analyses
2024,07200139547,PB,072003592,72247,2
2021,02700152100,PB,027000327,27354,2
2021,04400226978,PB,044000169,44151,2
2020,07200122045,PB,072003981,72022,2
2021,04400226978,PB,044000169,44030,2
2021,02700152100,PB,027000327,27313,2
2021,04400226978,PB,044000169,44210,2
2024,07200139547,PB,072003592,72257,2
2024,07200139547,PB,072003592,72360,2
2024,07200139547,PB,072003592,72003,2


--> Pas d'unicité sur certaines referenceprel pour PB.  
**Attention pour la création de la requête**

### Exemple de doublons
Pour un même prélèvement et une même commune.

In [8]:
%%sql

SELECT
    *
FROM
    int__resultats_udi_communes
WHERE
    de_partition = 2021
    AND categorie = 'metaux_lourds'
    AND cdparametresiseeaux = 'PB'
    AND referenceprel = '04400226978'
    AND cdreseau = '044000169'
    AND inseecommune = '44151'
;

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,de_partition,valtraduite,categorie,categorie_2,categorie_3,limite_qualite,valeur_sanitaire_1,valeur_sanitaire_2,cdreseau,inseecommune,datetimeprel
04400226978,PB,2021,0.000,metaux_lourds,None,None,10.0,None,None,044000169,44151,2021-11-16 11:40:00
04400226978,PB,2021,0.000,metaux_lourds,None,None,10.0,None,None,044000169,44151,2021-11-16 11:40:00


## Nombre de résultats Plomb chaque année, doublons compris (UDI et communes)

In [9]:
%%sql

WITH pb_prels AS (
    SELECT
        de_partition
    FROM
        'int__resultats_udi_communes'
    WHERE
        cdparametresiseeaux = 'PB'
    )

SELECT de_partition, COUNT(*)
FROM pb_prels
GROUP BY de_partition
;

Running query in 'duckdb:///../../database/data.duckdb'

de_partition,count_star()
2020,52653
2021,50380
2022,48543
2023,52014
2024,53140
2025,8332


## Test de suppression des doublons sur le cas précédent
Un prélèvement peut être sur plusieurs communes puisqu'une UDI peut desservir plusieurs communes.  
Ici on essaye de conserver la liste du prélèvement sans doublons pour une même commune.

In [10]:
%%sql --save pb_results_udi_dernier

WITH pb_prels AS (
    SELECT 
        *
    FROM
        'int__resultats_udi_communes'
    WHERE
        cdparametresiseeaux = 'PB'
),

deduplicated AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (
            PARTITION BY cdreseau, referenceprel, cdparametresiseeaux, datetimeprel, inseecommune
            ORDER BY cdreseau  -- arbitrary choice
        ) AS row_num
    FROM pb_prels
)

-- Obtention des résultats PB par UDI (cdreseau), avec suppression des doublons dus aux communes
SELECT *
FROM deduplicated
WHERE
    row_num = 1
    AND de_partition = 2021 
    AND categorie = 'metaux_lourds'
    AND cdparametresiseeaux = 'PB'
    AND referenceprel = '04400226978'
    AND cdreseau = '044000169'
    AND inseecommune = '44151'
;

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,de_partition,valtraduite,categorie,categorie_2,categorie_3,limite_qualite,valeur_sanitaire_1,valeur_sanitaire_2,cdreseau,inseecommune,datetimeprel,row_num
04400226978,PB,2021,0.000,metaux_lourds,None,None,10.0,None,None,044000169,44151,2021-11-16 11:40:00,1


--> le champ calculé row_num permet bien de supprimer les doublons sur les communes.

## Recherche du dernier prélèvement datant de moins d'un an
**Les doublons sont supprimés** par construction de la requête (prise en compte uniquement du dernier prélèvement pour une UDI)

### Test de suppression des doublons
On vérifie que les doublons sont bien supprimés ici quand on cherche le dernier prélèvement pour le Pb pour chaque UDI.

In [11]:
%%sql

WITH pb_dernier_prel AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        valtraduite,
        de_partition AS annee,
        limite_qualite,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        cdparametresiseeaux = 'PB'
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
),

pb_dernier_prel_check AS (
    SELECT
        cdreseau,
        COUNT(*) OVER (PARTITION BY cdreseau) AS nb_prel_udi
    FROM
        pb_dernier_prel
    WHERE
        row_number = 1
)

SELECT
    *
FROM
    pb_dernier_prel_check
WHERE
    nb_prel_udi > 1
;

Running query in 'duckdb:///../../database/data.duckdb'

cdreseau,nb_prel_udi


--> Aucune UDI avec plus d'un prélèvement de moins d'un an donc la CTE pb_dernier_prel est correcte

## Requête finale Pb

### Première version

In [12]:
%%sql

-- Ici on ne garde que le dernier prélèvement pour chaque UDI dans la dernière année
WITH pb_dernier_prel AS (
    SELECT
        int__resultats_udi_communes.*,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        cdparametresiseeaux = 'PB'
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
),

-- Ici on reprend les résultats pour chaque dernier prélèvement pour chaque UDI
resultats_udi_pb_dernier_prel AS (
    SELECT
        pb_dernier_prel.referenceprel,
        pb_dernier_prel.cdparametresiseeaux,
        pb_dernier_prel.valtraduite,
        pb_dernier_prel.categorie,
        pb_dernier_prel.limite_qualite,
        pb_dernier_prel.valeur_sanitaire_1,
        pb_dernier_prel.cdreseau,
        pb_dernier_prel.datetimeprel
    FROM
        pb_dernier_prel
    WHERE
        -- On ne garde que le dernier prélèvement
        -- pour chaque UDI (cdreseau)
        pb_dernier_prel.row_number = 1
)

-- Puis on ajoute le résultat pour chaque cas
SELECT
    resultats_udi_pb_dernier_prel.categorie,
    resultats_udi_pb_dernier_prel.cdreseau,
    resultats_udi_pb_dernier_prel.datetimeprel AS dernier_prel_datetime,
    'dernier_prel' AS periode,
    resultats_udi_pb_dernier_prel.valtraduite AS dernier_prel_valeur,
    CASE
        WHEN
            resultats_udi_pb_dernier_prel.valtraduite IS NULL
            OR resultats_udi_pb_dernier_prel.valtraduite = 0
            THEN 'aucun_parametre_quantifie'
        WHEN
            resultats_udi_pb_dernier_prel.valtraduite >= 10
            THEN 'sup_limite_qualite'
        WHEN
            resultats_udi_pb_dernier_prel.valtraduite >= 5
            AND resultats_udi_pb_dernier_prel.valtraduite < 10
            THEN 'sup_5_inf_10_limite_2036'
        WHEN
            resultats_udi_pb_dernier_prel.valtraduite < 5
            THEN 'inf_5'           
        ELSE 'erreur'
    END AS resultat
FROM
    resultats_udi_pb_dernier_prel
ORDER BY
    cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

categorie,cdreseau,dernier_prel_datetime,periode,dernier_prel_valeur,resultat
metaux_lourds,001000241,2025-02-06 09:21:00,dernier_prel,0.000,aucun_parametre_quantifie
metaux_lourds,001000250,2025-02-12 08:19:00,dernier_prel,0.000,aucun_parametre_quantifie
metaux_lourds,001000251,2025-02-12 08:19:00,dernier_prel,0.000,aucun_parametre_quantifie
metaux_lourds,001000252,2024-10-18 11:12:00,dernier_prel,7.000,sup_5_inf_10_limite_2036
metaux_lourds,001000253,2024-10-18 11:12:00,dernier_prel,7.000,sup_5_inf_10_limite_2036
metaux_lourds,001000258,2025-01-21 09:45:00,dernier_prel,0.000,aucun_parametre_quantifie
metaux_lourds,001000260,2024-08-22 10:46:00,dernier_prel,0.000,aucun_parametre_quantifie
metaux_lourds,001000269,2024-07-10 10:56:00,dernier_prel,3.000,inf_5
metaux_lourds,001000272,2024-08-22 08:47:00,dernier_prel,0.000,aucun_parametre_quantifie
metaux_lourds,001000275,2025-02-19 10:37:00,dernier_prel,0.000,aucun_parametre_quantifie


### Deuxième version (plus light)

In [13]:
%%sql

-- Ici on ne garde que le dernier prélèvement pour chaque UDI dans la dernière année
WITH pb_dernier_prel AS (
    SELECT
        int__resultats_udi_communes.*,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        cdparametresiseeaux = 'PB'
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
)

-- Ici on ne prend que le prélèvement le plus récent avec row_number = 1
SELECT
    categorie,
    cdreseau,
    datetimeprel AS dernier_prel_datetime,
    'dernier_prel' AS periode,
    valtraduite AS dernier_prel_valeur,
    CASE
        WHEN
            valtraduite IS NULL
            OR valtraduite = 0
            THEN 'aucun_parametre_quantifie'
        WHEN
            valtraduite >= 10
            THEN 'sup_limite_qualite'
        WHEN
            valtraduite >= 5
            AND valtraduite < 10
            THEN 'sup_limite_qualite_2036'
        WHEN
            valtraduite < 5
            THEN 'inf_5'           
        ELSE 'erreur'
    END AS resultat
FROM
    pb_dernier_prel
WHERE
    row_number = 1
ORDER BY
    cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

categorie,cdreseau,dernier_prel_datetime,periode,dernier_prel_valeur,resultat
metaux_lourds,001000241,2025-02-06 09:21:00,dernier_prel,0.000,aucun_parametre_quantifie
metaux_lourds,001000250,2025-02-12 08:19:00,dernier_prel,0.000,aucun_parametre_quantifie
metaux_lourds,001000251,2025-02-12 08:19:00,dernier_prel,0.000,aucun_parametre_quantifie
metaux_lourds,001000252,2024-10-18 11:12:00,dernier_prel,7.000,sup_limite_qualite_2036
metaux_lourds,001000253,2024-10-18 11:12:00,dernier_prel,7.000,sup_limite_qualite_2036
metaux_lourds,001000258,2025-01-21 09:45:00,dernier_prel,0.000,aucun_parametre_quantifie
metaux_lourds,001000260,2024-08-22 10:46:00,dernier_prel,0.000,aucun_parametre_quantifie
metaux_lourds,001000269,2024-07-10 10:56:00,dernier_prel,3.000,inf_5
metaux_lourds,001000272,2024-08-22 08:47:00,dernier_prel,0.000,aucun_parametre_quantifie
metaux_lourds,001000275,2025-02-19 10:37:00,dernier_prel,0.000,aucun_parametre_quantifie


### Vérif du changement de certaines valtraduite après avoir fait la V2 de la requête Pb dernier prél.

In [14]:
%%sql

select *
FROM
    int__resultats_udi_communes
WHERE
    cdreseau = '044000119'
    AND cdparametresiseeaux = 'PB'
    AND datetimeprel = '2024-11-06 10:10:00'
ORDER BY inseecommune

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,de_partition,valtraduite,categorie,categorie_2,categorie_3,limite_qualite,valeur_sanitaire_1,valeur_sanitaire_2,cdreseau,inseecommune,datetimeprel
04400259700,PB,2024,0.200,metaux_lourds,None,None,10.0,None,None,044000119,44003,2024-11-06 10:10:00
04400259369,PB,2024,0.600,metaux_lourds,None,None,10.0,None,None,044000119,44003,2024-11-06 10:10:00
04400259700,PB,2024,0.200,metaux_lourds,None,None,10.0,None,None,044000119,44048,2024-11-06 10:10:00
04400259369,PB,2024,0.600,metaux_lourds,None,None,10.0,None,None,044000119,44048,2024-11-06 10:10:00
04400259700,PB,2024,0.200,metaux_lourds,None,None,10.0,None,None,044000119,44096,2024-11-06 10:10:00
04400259369,PB,2024,0.600,metaux_lourds,None,None,10.0,None,None,044000119,44096,2024-11-06 10:10:00
04400259700,PB,2024,0.200,metaux_lourds,None,None,10.0,None,None,044000119,44104,2024-11-06 10:10:00
04400259369,PB,2024,0.600,metaux_lourds,None,None,10.0,None,None,044000119,44104,2024-11-06 10:10:00
04400259700,PB,2024,0.200,metaux_lourds,None,None,10.0,None,None,044000119,44107,2024-11-06 10:10:00
04400259369,PB,2024,0.600,metaux_lourds,None,None,10.0,None,None,044000119,44107,2024-11-06 10:10:00


--> Il y a des relevés avec des referenceprel différentes mais qui ont lieu au même moment et qui ont des valtraduite différentes.  
Ca crée un **problème quand on filtre avec row_number pour obtenir le dernier prélèvement** : comme ils ont lieu au même moment pour une UDI (cdreseau), une valeur parmi les doublons est choisie au hasard.  

--> Pour l'instant on garde cette requête qu'on reproduit sur l'Arsenic. **Il faudra peut-être la modifier en fonction des retours**.

# Requête finale PB + AS - dernier prélèvement
Créée après ajout de AS : remplace la requête finale Pb ci-dessus

In [27]:
%%sql

-- Ici on ne garde que le dernier prélèvement
-- pour chaque UDI dans la dernière année
WITH metaux_lourds_dernier_prel AS (
    SELECT
        *,
        ROW_NUMBER()
            OVER (
                PARTITION BY
                    cdreseau,
                    cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        cdparametresiseeaux IN ('PB', 'AS')
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel
        < INTERVAL 1 YEAR
)

-- Ici on ne prend que le prélèvement le plus récent (avec row_number = 1)
-- pour chaque type de métaux lourds
SELECT
    cdreseau,
    datetimeprel AS dernier_prel_datetime,
    'dernier_prel' AS periode,
    valtraduite AS dernier_prel_valeur,
    CASE
        WHEN
            cdparametresiseeaux = 'PB'
            THEN 'metaux_lourds_pb'
        WHEN
            cdparametresiseeaux = 'AS'
            THEN 'metaux_lourds_as'
    END AS categorie,
    CASE
        WHEN
            -- Pas de distinction PB/AS car même résultat
            valtraduite IS NULL
            OR valtraduite = 0
            THEN 'non_quantifie'
        WHEN
            cdparametresiseeaux = 'PB'
            AND valtraduite >= limite_qualite
            THEN 'sup_limite_qualite'
        WHEN
            -- 5 est la future limite de qualité appliquée
            -- à partir de 2036
            cdparametresiseeaux = 'PB'
            AND valtraduite >= 5
            AND valtraduite < limite_qualite
            THEN 'sup_limite_qualite_2036'
        WHEN
            cdparametresiseeaux = 'PB'
            AND valtraduite < 5
            THEN 'inf_5'
        WHEN
            cdparametresiseeaux = 'AS'
            AND valtraduite >= valeur_sanitaire_1
            THEN 'sup_limite_sanitaire'
        WHEN
            cdparametresiseeaux = 'AS'
            AND valtraduite >= limite_qualite
            AND valtraduite < valeur_sanitaire_1
            THEN 'sup_limite_qualite'
        WHEN
            cdparametresiseeaux = 'AS'
            AND valtraduite < limite_qualite
            THEN 'inf_limite_qualite'
        ELSE 'erreur'
    END AS resultat
FROM
    metaux_lourds_dernier_prel
WHERE
    row_number = 1
ORDER BY
    cdreseau


Running query in 'duckdb:///../../database/data.duckdb'

cdreseau,dernier_prel_datetime,periode,dernier_prel_valeur,categorie,resultat
001000235,2024-12-13 09:36:00,dernier_prel,0.000,metaux_lourds_as,non_quantifie
001000241,2025-02-06 09:21:00,dernier_prel,0.000,metaux_lourds_pb,non_quantifie
001000241,2025-02-10 11:04:00,dernier_prel,0.000,metaux_lourds_as,non_quantifie
001000248,2024-11-25 09:02:00,dernier_prel,0.000,metaux_lourds_as,non_quantifie
001000249,2024-10-18 09:09:00,dernier_prel,0.000,metaux_lourds_as,non_quantifie
001000250,2025-01-21 10:40:00,dernier_prel,0.000,metaux_lourds_as,non_quantifie
001000250,2025-02-12 08:19:00,dernier_prel,0.000,metaux_lourds_pb,non_quantifie
001000251,2025-02-12 08:19:00,dernier_prel,0.000,metaux_lourds_pb,non_quantifie
001000252,2024-10-18 11:12:00,dernier_prel,7.000,metaux_lourds_pb,sup_limite_qualite_2036
001000252,2024-06-11 08:33:00,dernier_prel,0.000,metaux_lourds_as,non_quantifie


# Construction requête UDI prélèvement annuel

In [16]:
%%sql

WITH
metaux_lourds_prels AS (
    -- Certains prélèvements ont plusieurs analyses pour la même substance
    -- Le SELECT DISTINCT ne dédoublonne pas ces cas là
    -- Donc il n'y a pas d'unicité sur referenceprel dans cetre requête
    SELECT DISTINCT
        de_partition AS annee,
        cdreseau,
        cdparametresiseeaux,
        valeur_sanitaire_1,
        limite_qualite,
        referenceprel,
        datetimeprel,
        valtraduite
    FROM
        'int__resultats_udi_communes'
    WHERE
        cdparametresiseeaux IN ('PB', 'AS')
)

SELECT
    cdreseau,
    annee,
    CASE
        WHEN cdparametresiseeaux = 'PB' THEN 'metaux_lourds_pb'
        WHEN cdparametresiseeaux = 'AS' THEN 'metaux_lourds_as'
    END AS categorie,
    'bilan_annuel_' || annee AS periode,
    count(
        DISTINCT
        CASE
            WHEN
                cdparametresiseeaux = 'PB'
                AND valtraduite IS NOT NULL AND valtraduite >= limite_qualite
                THEN referenceprel
            WHEN
                cdparametresiseeaux = 'AS'
                AND valtraduite IS NOT NULL AND valtraduite >= valeur_sanitaire_1
                THEN referenceprel
        END
    ) AS nb_depassements,
    count(DISTINCT referenceprel) AS nb_prelevements,
    (
        count(
            DISTINCT
            CASE
                WHEN
                    cdparametresiseeaux = 'PB'
                    AND valtraduite IS NOT NULL
                    AND valtraduite >= limite_qualite
                    THEN referenceprel
                WHEN
                    cdparametresiseeaux = 'AS'
                    AND valtraduite IS NOT NULL
                    AND valtraduite >= valeur_sanitaire_1
                    THEN referenceprel
            END
        )::float
        /
        count(DISTINCT referenceprel)::float
    ) AS ratio_limite_sanitaire

FROM metaux_lourds_prels

GROUP BY cdreseau, annee, categorie

Running query in 'duckdb:///../../database/data.duckdb'

cdreseau,annee,categorie,periode,nb_depassements,nb_prelevements,ratio_limite_sanitaire
060000778,2023,metaux_lourds_as,bilan_annuel_2023,0,3,0.0
037000202,2020,metaux_lourds_as,bilan_annuel_2020,0,1,0.0
034001615,2022,metaux_lourds_pb,bilan_annuel_2022,0,1,0.0
083000029,2020,metaux_lourds_as,bilan_annuel_2020,0,3,0.0
038001363,2021,metaux_lourds_as,bilan_annuel_2021,0,1,0.0
056000152,2021,metaux_lourds_as,bilan_annuel_2021,0,6,0.0
018000633,2022,metaux_lourds_as,bilan_annuel_2022,0,4,0.0
009001410,2021,metaux_lourds_as,bilan_annuel_2021,0,4,0.0
060000936,2021,metaux_lourds_pb,bilan_annuel_2021,0,2,0.0
051000436,2022,metaux_lourds_as,bilan_annuel_2022,0,1,0.0


# Tests pour identifier l'origine des doublons
Le test d'unicité inseecommune/annee bug

In [17]:
%%sql

WITH
metaux_lourds_prels AS (
    -- Certains prélèvements ont plusieurs analyses pour la même substance
    -- Le SELECT DISTINCT ne dédoublonne pas ces cas là
    -- Donc il n'y a pas d'unicité sur referenceprel dans cetre requête
    SELECT DISTINCT
        de_partition AS annee,
        inseecommune,
        cdparametresiseeaux,
        valeur_sanitaire_1,
        limite_qualite,
        referenceprel,
        datetimeprel,
        valtraduite
    FROM
        int__resultats_udi_communes
    WHERE
        cdparametresiseeaux IN ('PB', 'AS')
)

SELECT
    inseecommune,
    annee,
    CASE
        WHEN cdparametresiseeaux = 'PB' THEN 'metaux_lourds_pb'
        WHEN cdparametresiseeaux = 'AS' THEN 'metaux_lourds_as'
    END AS categorie,
    'bilan_annuel_' || annee AS periode,
    count(
        DISTINCT
        CASE
            WHEN
                cdparametresiseeaux = 'PB'
                AND valtraduite IS NOT NULL
                AND valtraduite >= limite_qualite
                THEN referenceprel
            WHEN
                cdparametresiseeaux = 'AS'
                AND valtraduite IS NOT NULL
                AND valtraduite >= valeur_sanitaire_1
                THEN referenceprel
        END
    ) AS nb_depassements,
    count(DISTINCT referenceprel) AS nb_prelevements,
    (
        count(
            DISTINCT
            CASE
                WHEN
                    cdparametresiseeaux = 'PB'
                    AND valtraduite IS NOT NULL
                    AND valtraduite >= limite_qualite
                    THEN referenceprel
                WHEN
                    cdparametresiseeaux = 'AS'
                    AND valtraduite IS NOT NULL
                    AND valtraduite >= valeur_sanitaire_1
                    THEN referenceprel
            END
        )::float
        /
        count(DISTINCT referenceprel)::float
    ) AS ratio_limite_sanitaire

FROM metaux_lourds_prels

WHERE inseecommune = '01001' and annee = '2020'

GROUP BY inseecommune, annee, categorie


Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,annee,categorie,periode,nb_depassements,nb_prelevements,ratio_limite_sanitaire
01001,2020,metaux_lourds_as,bilan_annuel_2020,0,3,0.0
01001,2020,metaux_lourds_pb,bilan_annuel_2020,0,3,0.0


In [18]:
%%sql

WITH
prels AS (
    -- Certains prélèvements ont plusieurs analyses pour la même substance
    -- Le SELECT DISTINCT ne dédoublonne pas ces cas là
    -- Donc il n'y a pas d'unicité sur referenceprel dans cetre requête
    SELECT DISTINCT
        de_partition AS annee,
        inseecommune,
        cdparametresiseeaux,
        valeur_sanitaire_1,
        referenceprel,
        datetimeprel,
        valtraduite
    FROM
        int__resultats_udi_communes
    WHERE
        cdparametresiseeaux IN ('14DAN', 'PCLAT')
)

SELECT
    inseecommune,
    annee,
    CASE
        WHEN cdparametresiseeaux = '14DAN' THEN 'sub_indus_14dioxane'
        WHEN cdparametresiseeaux = 'PCLAT' THEN 'sub_indus_perchlorate'
    END AS categorie,
    'bilan_annuel_' || annee AS periode,
    count(
        DISTINCT
        CASE
            WHEN
                valtraduite IS NOT NULL AND valtraduite >= valeur_sanitaire_1
                THEN referenceprel
        END
    ) AS nb_depassements,
    count(DISTINCT referenceprel) AS nb_prelevements,
    (
        count(
            DISTINCT
            CASE
                WHEN
                    valtraduite IS NOT NULL
                    AND valtraduite >= valeur_sanitaire_1
                    THEN referenceprel
            END
        )::float
        /
        count(DISTINCT referenceprel)::float
    ) AS ratio_limite_sanitaire

FROM prels

GROUP BY inseecommune, annee, categorie

ORDER BY inseecommune, annee

Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,annee,categorie,periode,nb_depassements,nb_prelevements,ratio_limite_sanitaire
01173,2020,sub_indus_14dioxane,bilan_annuel_2020,0,2,0.0
01247,2020,sub_indus_14dioxane,bilan_annuel_2020,0,2,0.0
02001,2020,sub_indus_perchlorate,bilan_annuel_2020,0,1,0.0
02001,2021,sub_indus_perchlorate,bilan_annuel_2021,0,1,0.0
02001,2022,sub_indus_perchlorate,bilan_annuel_2022,0,1,0.0
02001,2023,sub_indus_perchlorate,bilan_annuel_2023,0,1,0.0
02002,2020,sub_indus_perchlorate,bilan_annuel_2020,1,1,1.0
02002,2021,sub_indus_perchlorate,bilan_annuel_2021,1,1,1.0
02002,2022,sub_indus_perchlorate,bilan_annuel_2022,1,1,1.0
02002,2023,sub_indus_perchlorate,bilan_annuel_2023,0,1,0.0


In [19]:
%%sql

SELECT DISTINCT
    de_partition AS annee,
    inseecommune,
    cdparametresiseeaux,
    valeur_sanitaire_1,
    referenceprel,
    datetimeprel,
    valtraduite
FROM
    int__resultats_udi_communes
WHERE
    cdparametresiseeaux IN ('PB', 'AS')
    and inseecommune = '01001'
    and de_partition = 2020
ORDER BY
    referenceprel

Running query in 'duckdb:///../../database/data.duckdb'

annee,inseecommune,cdparametresiseeaux,valeur_sanitaire_1,referenceprel,datetimeprel,valtraduite
2020,01001,AS,13.0,00100119863,2020-02-21 09:43:00,0.000
2020,01001,PB,None,00100121030,2020-06-26 09:25:00,0.000
2020,01001,AS,13.0,00100121163,2020-06-26 08:59:00,0.000
2020,01001,PB,None,00100121499,2020-07-17 09:50:00,3.000
2020,01001,PB,None,00100121500,2020-07-17 09:05:00,0.000
2020,01001,AS,13.0,00100124041,2020-12-17 09:59:00,0.000


In [20]:
%%sql

SELECT DISTINCT
    de_partition AS annee,
    inseecommune,
    cdparametresiseeaux,
    valeur_sanitaire_1,
    referenceprel,
    datetimeprel,
    valtraduite,
FROM
    int__resultats_udi_communes
WHERE
    cdparametresiseeaux IN ('14DAN', 'PCLAT')
    AND inseecommune = '33063'
    and de_partition = 2024


Running query in 'duckdb:///../../database/data.duckdb'

annee,inseecommune,cdparametresiseeaux,valeur_sanitaire_1,referenceprel,datetimeprel,valtraduite
2024,33063,PCLAT,4.0,03300258152,2024-01-10 10:55:00,0.700
2024,33063,PCLAT,4.0,03300264695,2024-09-23 11:20:00,0.000
2024,33063,PCLAT,4.0,03300264696,2024-09-23 12:00:00,0.000
2024,33063,PCLAT,4.0,03300265349,2024-10-16 11:55:00,0.000
2024,33063,PCLAT,4.0,03300265354,2024-10-24 09:20:00,0.000
2024,33063,PCLAT,4.0,03300266363,2024-12-23 10:20:00,0.000
2024,33063,PCLAT,4.0,03300264457,2024-08-28 07:35:00,0.000
2024,33063,PCLAT,4.0,03300258459,2024-01-24 08:55:00,0.600
2024,33063,PCLAT,4.0,03300258748,2024-02-13 10:45:00,0.000
2024,33063,PCLAT,4.0,03300264795,2024-09-23 11:50:00,0.600
